In [191]:
# ！pip install functorch
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import pandas as pd
import scipy
from scipy import stats
import numpy as np
from os import path
import torchvision

In [192]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# print(torch.cuda.get_device_name())
print(f"Using {device} device")

Using cpu device


In [193]:
MA_DAYS = 25
trading_days_in_year = 252

# Import raw data from yahoo finance

In [194]:
from google.colab import drive
drive.mount('/content/drive')
data_files_path_prefix = "/content/drive/MyDrive"
data_files_path = "ML-Portfolio-Data"
data_files_path = path.join(data_files_path_prefix, data_files_path)

high_risk_file = 'SPY.csv'
low_risk_file = 'IEF.csv'
high_risk = pd.read_csv(path.join(data_files_path, high_risk_file))
low_risk = pd.read_csv(path.join(data_files_path, low_risk_file))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [195]:
# Read files from the same directory
#high_risk = pd.read_csv('SPY.csv')
#low_risk = pd.read_csv('O9P.SI.csv')

In [196]:
# high_risk = high_risk[:1008]
# low_risk = low_risk[:1008]
print(high_risk.shape)
print(low_risk.shape)

(5147, 7)
(5147, 7)


In [197]:
high_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-07-30,89.320000,91.400002,88.720001,90.940002,61.380939,47532200
1,2002-07-31,90.489998,91.550003,89.250000,91.160004,61.529453,44669900
2,2002-08-01,90.879997,91.349998,88.330002,88.779999,59.923054,66571900
3,2002-08-02,88.500000,88.910004,85.620003,86.790001,58.579895,51772900
4,2002-08-05,86.489998,86.930000,83.550003,83.769997,56.541496,47191300


In [198]:
low_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-07-30,81.940002,82.120003,81.699997,81.769997,45.672558,41300
1,2002-07-31,82.050003,82.580002,82.050003,82.519997,46.091442,32600
2,2002-08-01,82.540001,82.900002,82.519997,82.860001,46.281376,71400
3,2002-08-02,83.019997,83.699997,82.900002,83.500000,46.638828,120300
4,2002-08-05,83.680000,83.919998,83.529999,83.919998,46.873459,159300


# ML Portfolio

In [199]:
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset

## Enrich data

### Calculate daily returns

In [200]:
def add_daily_return(market_data):
    market_data["Daily Return"]  = market_data['Close'] - market_data['Open']

add_daily_return(high_risk)
add_daily_return(low_risk)

### Calculate moving average (MA) of daily returns

In [201]:
def add_moving_average(market_data, ma_days):
    temp_vars = []

    # df = market_data
    for i in range(0,ma_days):
        temp_var = "M_{0}".format(i)
        market_data[temp_var] = market_data["Daily Return"].shift(i)
        temp_vars.append(temp_var)

    market_data["MA"] = market_data[temp_vars].mean(axis=1)

    for i in range(0,ma_days):
        temp_var = "M_{0}".format(i)
        market_data.drop(temp_var, axis = 1, inplace = True)

add_moving_average(high_risk, MA_DAYS)
add_moving_average(low_risk, MA_DAYS)


In [202]:
high_risk.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA
5142,2022-12-30,380.640015,382.579987,378.429993,382.429993,382.429993,83975100,1.789978,-0.368799
5143,2023-01-03,384.369995,386.429993,377.829987,380.820007,380.820007,74850700,-3.549988,-0.530798
5144,2023-01-04,383.179993,385.880005,380.000000,383.760010,383.760010,85934100,0.580017,-0.380398
5145,2023-01-05,381.720001,381.839996,378.760010,379.380005,379.380005,76970500,-2.339996,-0.441199
5146,2023-01-06,382.609985,389.250000,379.410004,388.079987,388.079987,104041300,5.470002,-0.709999


In [203]:
low_risk.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA
5142,2022-12-30,95.860001,96.269997,95.620003,95.779999,95.779999,5039800,-0.080002,0.050399
5143,2023-01-03,96.910004,97.000000,96.339996,96.529999,96.529999,6808300,-0.380005,0.025599
5144,2023-01-04,97.339996,97.419998,96.989998,97.269997,97.269997,7800100,-0.069999,0.025599
5145,2023-01-05,96.699997,97.220001,96.570000,97.129997,97.129997,3177900,0.430000,0.043600
5146,2023-01-06,97.169998,98.430000,97.080002,98.379997,98.379997,6807700,1.209999,0.050399


### Calculate ROE

In [204]:
def add_roe(market_data):    
    market_data["Next Close"] = market_data["Close"].shift(-1)
    market_data["ROE"] = (market_data["Next Close"] - market_data["Close"]) / market_data['Close']

add_roe(high_risk)
add_roe(low_risk)

In [205]:
def add_roe_binary(market_data, tau=-0.005):    
    market_data["ROE Binary"] = np.where(market_data["ROE"].values < tau, 0, 1)

add_roe_binary(high_risk)
add_roe_binary(low_risk)

In [206]:
high_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA,Next Close,ROE,ROE Binary
0,2002-07-30,89.320000,91.400002,88.720001,90.940002,61.380939,47532200,1.620002,1.620002,91.160004,0.002419,1
1,2002-07-31,90.489998,91.550003,89.250000,91.160004,61.529453,44669900,0.670006,1.145004,88.779999,-0.026108,0
2,2002-08-01,90.879997,91.349998,88.330002,88.779999,59.923054,66571900,-2.099998,0.063337,86.790001,-0.022415,0
3,2002-08-02,88.500000,88.910004,85.620003,86.790001,58.579895,51772900,-1.709999,-0.379997,83.769997,-0.034797,0
4,2002-08-05,86.489998,86.930000,83.550003,83.769997,56.541496,47191300,-2.720001,-0.847998,86.589996,0.033664,1


In [207]:
low_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA,Next Close,ROE,ROE Binary
0,2002-07-30,81.940002,82.120003,81.699997,81.769997,45.672558,41300,-0.170005,-0.170005,82.519997,0.009172,1
1,2002-07-31,82.050003,82.580002,82.050003,82.519997,46.091442,32600,0.469994,0.149994,82.860001,0.004120,1
2,2002-08-01,82.540001,82.900002,82.519997,82.860001,46.281376,71400,0.320000,0.206663,83.500000,0.007724,1
3,2002-08-02,83.019997,83.699997,82.900002,83.500000,46.638828,120300,0.480003,0.274998,83.919998,0.005030,1
4,2002-08-05,83.680000,83.919998,83.529999,83.919998,46.873459,159300,0.239998,0.267998,83.239998,-0.008103,0


## Build feature space

In [208]:
def remove_for_ma(market_data, ma_days):
  return market_data[ma_days:]

high_risk = remove_for_ma(high_risk, MA_DAYS)
low_risk = remove_for_ma(low_risk, MA_DAYS)

In [209]:
print(high_risk.shape)

(5122, 12)


In [210]:
def standardize_columns(market_data, columns):
  for column in columns:
    market_data[column] = market_data[column]/market_data[column].std()

standardize_columns(high_risk, ['Volume', 'Daily Return', 'MA'])
standardize_columns(low_risk, ['Volume', 'Daily Return', 'MA'])

In [211]:
high_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA,Next Close,ROE,ROE Binary
25,2002-09-04,88.610001,90.250000,88.059998,89.540001,60.436001,0.550024,0.479605,0.276737,88.779999,-0.008488,0
26,2002-09-05,88.489998,89.430000,87.500000,88.779999,59.923054,0.723874,0.149555,0.229367,90.000000,0.013742,1
27,2002-09-06,89.750000,90.570000,89.339996,90.000000,60.746498,0.415721,0.128926,0.522307,90.660004,0.007333,1
28,2002-09-09,89.099998,91.349998,88.800003,90.660004,61.191929,0.365951,0.804501,0.929931,91.699997,0.011471,1
29,2002-09-10,91.139999,91.779999,90.559998,91.699997,61.893936,0.445799,0.288793,1.338801,91.129997,-0.006216,0


In [212]:
low_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA,Next Close,ROE,ROE Binary
25,2002-09-04,85.160004,85.449997,85.080002,85.199997,47.752071,0.023505,0.135391,0.912126,85.540001,0.003991,1
26,2002-09-05,85.599998,85.650002,85.190002,85.540001,47.942638,0.017606,-0.203112,0.564337,84.879997,-0.007716,0
27,2002-09-06,85.089996,85.250000,84.839996,84.879997,47.572742,0.009791,-0.710926,0.216542,84.760002,-0.001414,1
28,2002-09-09,84.940002,85.150002,84.750000,84.760002,47.505463,0.027002,-0.609368,-0.216563,85.059998,0.003539,1
29,2002-09-10,84.709999,85.209999,84.660004,85.059998,47.673588,0.006507,1.184878,-0.144378,84.750000,-0.003644,1


In [213]:
# def to_dataset(low_risk, high_risk):
#   return np.vstack((low_risk['Daily Return'], low_risk['MA'], low_risk['Volume'], high_risk['Daily Return'], high_risk['MA'], high_risk['Volume'],high_risk['ROE Binary']))

# dataset = to_dataset(low_risk, high_risk).T
# print(dataset.shape, dataset)

In [214]:
# pd.concat([low_risk, high_risk], join='outer', axis=1)[['Date'],['Daily Return'],['MA'],['Volume'],['ROE Binary']]
# pd.concat([low_risk.add_prefix('l_'), high_risk.add_prefix('h_')], join='outer', axis=1)[['Date','ROE Binary']]
master_dataset = pd.concat([low_risk.add_prefix('l_'), high_risk.add_prefix('h_')], join='outer', axis=1)[['l_Date','l_Daily Return','l_MA','l_Volume','h_Daily Return','h_MA','h_Volume','h_ROE Binary']]
master_dataset

,l_Date,l_Daily Return,l_MA,l_Volume,h_Daily Return,h_MA,h_Volume,h_ROE Binary
25,2002-09-04,0.135391,0.912126,0.023505,0.479605,0.276737,0.550024,0
26,2002-09-05,-0.203112,0.564337,0.017606,0.149555,0.229367,0.723874,1
27,2002-09-06,-0.710926,0.216542,0.009791,0.128926,0.522307,0.415721,1
28,2002-09-09,-0.609368,-0.216563,0.027002,0.804501,0.929931,0.365951,1
29,2002-09-10,1.184878,-0.144378,0.006507,0.288793,1.338801,0.445799,0
...,...,...,...,...,...,...,...,...
5142,2022-12-30,-0.270837,0.826825,1.532486,0.923099,-1.149319,0.903889,1
5143,2023-01-03,-1.286460,0.419968,2.070246,-1.830743,-1.654172,0.805676,1
5144,2023-01-04,-0.236973,0.419968,2.371829,0.299117,-1.185467,0.924976,0
5145,2023-01-05,1.455712,0.715269,0.966325,-1.206745,-1.374945,0.828493,1


In [215]:
# X_tensor = torch.from_numpy(master_dataset[:,:-1])
# Y_tensor = torch.from_numpy(master_dataset[:,-1])

## Build graph

In [216]:
lr = 1e-1
n_epochs = 500
torch.manual_seed(42)
lambda1 = 1e-3 #0.5
lambda2 = 1e-3 #0.5

loss_fn = nn.BCELoss()

batch_size = 32

In [217]:
folds=10
splits=KFold(n_splits=folds,shuffle=True,random_state=42)

In [218]:
#no cross-validation

def train_and_get_a_b(dataset):

  a = torch.randn((6), requires_grad=True, dtype=torch.double)
  b = torch.randn((6), requires_grad=True, dtype=torch.double)
  # print(a, a.size(), b, b.size())

  optimizer = optim.SGD([a, b], lr=lr)

  X_tensor = torch.from_numpy(dataset[:,:-1])
  Y_tensor = torch.from_numpy(dataset[:,-1])
  # print(X_tensor, Y_tensor)
    
    
  for epoch in range(n_epochs):
    
      yhat = torch.exp(torch.matmul(X_tensor, a)) / (torch.exp (torch.matmul(X_tensor, a)) + torch.exp(torch.matmul(X_tensor, b)))

      loss = loss_fn(yhat, Y_tensor)
      loss.backward()   

      if epoch % 10 == 0:
        print(f"Epoch: {epoch}. Loss: {loss}")

      optimizer.step()
      optimizer.zero_grad()
      
  return a,b

## Backtesting

In [219]:
import time
from datetime import date
from datetime import timedelta

In [220]:
first_date = date(2003,9,21)
last_date = date(2023,1,1)

In [221]:
delta_50weeks = timedelta(weeks=50)
delta_1week = timedelta(weeks=1)

In [222]:
daterange = pd.date_range(first_date, last_date, freq='1W')
daterange

DatetimeIndex(['2003-09-21', '2003-09-28', '2003-10-05', '2003-10-12',
               '2003-10-19', '2003-10-26', '2003-11-02', '2003-11-09',
               '2003-11-16', '2003-11-23',
               ...
               '2022-10-30', '2022-11-06', '2022-11-13', '2022-11-20',
               '2022-11-27', '2022-12-04', '2022-12-11', '2022-12-18',
               '2022-12-25', '2023-01-01'],
              dtype='datetime64[ns]', length=1007, freq='W-SUN')

In [238]:
def get_dataset_for_date(date):
  startdate = pd.to_datetime(date) - delta_50weeks
  enddate = pd.to_datetime(date)
  mask = (pd.to_datetime(master_dataset['l_Date']) > startdate) & (pd.to_datetime(master_dataset['l_Date']) <= enddate)
  subset = master_dataset.loc[mask]
  # print(subset)
  dataset = subset[['l_Daily Return','l_MA','l_Volume','h_Daily Return','h_MA','h_Volume','h_ROE Binary']].to_numpy()
  # print(dataset)
  return dataset

In [239]:
dataset = get_dataset_for_date(first_date)
dataset[:-1]

array([[ 0.10155789,  1.89647755,  0.02441736, ..., -0.97480731,
         0.57250387,  1.        ],
       [ 0.40625526,  1.88335059,  0.04092873, ..., -0.59959326,
         0.8560541 ,  0.        ],
       [ 0.16925525,  1.88991473,  0.02873525, ..., -0.83893149,
         0.86063301,  1.        ],
       ...,
       [ 0.20311239,  0.702162  ,  0.02435655, ...,  0.53227856,
         0.40786757,  1.        ],
       [ 1.11718079,  1.03027115,  0.05385199, ...,  0.41510118,
         0.3432117 ,  1.        ],
       [-0.06772444,  1.33868853,  0.04019894, ...,  0.66191837,
         0.32553542,  0.        ]])

In [240]:
a,b = train_and_get_a_b(dataset[:-1])

Epoch: 0. Loss: 0.7215387084732557
Epoch: 10. Loss: 0.6810488372594536
Epoch: 20. Loss: 0.6619500050935622
Epoch: 30. Loss: 0.6521565567197636
Epoch: 40. Loss: 0.6464397969752089
Epoch: 50. Loss: 0.6426307457433627
Epoch: 60. Loss: 0.6398216117779618
Epoch: 70. Loss: 0.6376153153449661
Epoch: 80. Loss: 0.6358225985627264
Epoch: 90. Loss: 0.6343410710924327
Epoch: 100. Loss: 0.6331068216799932
Epoch: 110. Loss: 0.6320747596039019
Epoch: 120. Loss: 0.6312103348754365
Epoch: 130. Loss: 0.6304858042589387
Epoch: 140. Loss: 0.629878352162473
Epoch: 150. Loss: 0.6293690012873694
Epoch: 160. Loss: 0.6289418863431444
Epoch: 170. Loss: 0.6285837156714381
Epoch: 180. Loss: 0.6282833454216006
Epoch: 190. Loss: 0.6280314310362333
Epoch: 200. Loss: 0.6278201373411659
Epoch: 210. Loss: 0.6276428956928688
Epoch: 220. Loss: 0.6274942000225749
Epoch: 230. Loss: 0.6273694354503347
Epoch: 240. Loss: 0.6272647343062115
Epoch: 250. Loss: 0.6271768552427145
Epoch: 260. Loss: 0.6271030817955745
Epoch: 270. L

In [241]:
with torch.no_grad():
  y_test = torch.exp(torch.matmul(torch.from_numpy(dataset[-1][:-1]), a)) / (torch.exp (torch.matmul(torch.from_numpy(dataset[-1][:-1]), a)) + torch.exp(torch.matmul(torch.from_numpy(dataset[-1][:-1]), b)))
  print(y_test)

tensor(0.7145, dtype=torch.float64)


In [242]:
def calculate_ml_portfolio_weights(x, k):
  return 0 if x < k else 1

In [243]:
weight = calculate_ml_portfolio_weights(y_test.numpy(), 0.5)
weight

1

In [244]:
first_date

datetime.date(2003, 9, 21)

In [245]:
def get_backtest_data(date, weight):
  startdate = pd.to_datetime(date)
  enddate = pd.to_datetime(date) + delta_1week

  investment = low_risk if weight == 0 else high_risk
    
  backtest_mask = (pd.to_datetime(investment['Date']) > startdate) & (pd.to_datetime(investment['Date']) <= enddate)
  backtest_data = investment.loc[backtest_mask]

  return backtest_data

In [246]:
def calculate_backtest_return(backtest_data):
  first_open = backtest_data.iloc[0]['Open']
  last_close = backtest_data.iloc[-1]['Close']
  return (last_close - first_open)/first_open

In [247]:
def get_backtest_return(date, weight):
  backtest_data = get_backtest_data(date, weight)
  return calculate_backtest_return(backtest_data)

In [248]:
backtest_data = get_backtest_data(first_date, weight)
backtest_data.iloc[-1]['Close']
backtest_data.iloc[0]['Open']

102.849998

In [249]:
get_backtest_return(first_date, weight)

-0.028196412799152443

In [250]:
if weight == 0:
  print(low_risk.loc[low_risk['Date'] == first_date])
elif weight == 1:
  pass

In [251]:
backtest_returns = {}

for date in daterange:
  print(date)
  dataset = get_dataset_for_date(date)
  a,b = train_and_get_a_b(dataset[:-1])
  with torch.no_grad():
    y_test = torch.exp(torch.matmul(torch.from_numpy(dataset[-1][:-1]), a)) / (torch.exp (torch.matmul(torch.from_numpy(dataset[-1][:-1]), a)) + torch.exp(torch.matmul(torch.from_numpy(dataset[-1][:-1]), b)))
    print(y_test)
  weight = calculate_ml_portfolio_weights(y_test.numpy(), 0.5)
  ret = get_backtest_return(date, weight)
  backtest_returns[date] = ret

Streaming output truncated to the last 5000 lines.
Epoch: 430. Loss: 0.5792344553998144
Epoch: 440. Loss: 0.5792179909400019
Epoch: 450. Loss: 0.5792041471138083
Epoch: 460. Loss: 0.579192504572068
Epoch: 470. Loss: 0.579182711579611
Epoch: 480. Loss: 0.5791744729962807
Epoch: 490. Loss: 0.5791675410806006
tensor(0.9888, dtype=torch.float64)
2021-03-07 00:00:00
Epoch: 0. Loss: 3.700318248437614
Epoch: 10. Loss: 1.6217001995774516
Epoch: 20. Loss: 0.8523053443053644
Epoch: 30. Loss: 0.6768278306758663
Epoch: 40. Loss: 0.6274131517005561
Epoch: 50. Loss: 0.6074411111519081
Epoch: 60. Loss: 0.5979778990245317
Epoch: 70. Loss: 0.5927103496028076
Epoch: 80. Loss: 0.5893202258259215
Epoch: 90. Loss: 0.5869193046071116
Epoch: 100. Loss: 0.5851289847961778
Epoch: 110. Loss: 0.5837597132965624
Epoch: 120. Loss: 0.582699265561636
Epoch: 130. Loss: 0.5818722158937859
Epoch: 140. Loss: 0.5812239993128596
Epoch: 150. Loss: 0.5807137246473957
Epoch: 160. Loss: 0.5803102895203948
Epoch: 170. Loss: 0.

In [252]:
backtest_returns

{Timestamp('2003-09-21 00:00:00', freq='W-SUN'): -0.028196412799152443,
 Timestamp('2003-09-28 00:00:00', freq='W-SUN'): 0.030807536466374772,
 Timestamp('2003-10-05 00:00:00', freq='W-SUN'): 0.010533407116348816,
 Timestamp('2003-10-12 00:00:00', freq='W-SUN'): -0.004297555193367887,
 Timestamp('2003-10-19 00:00:00', freq='W-SUN'): -0.008329296553258906,
 Timestamp('2003-10-26 00:00:00', freq='W-SUN'): 0.015037642472289269,
 Timestamp('2003-11-02 00:00:00', freq='W-SUN'): -0.0013238676122931735,
 Timestamp('2003-11-09 00:00:00', freq='W-SUN'): -0.002647995132362341,
 Timestamp('2003-11-16 00:00:00', freq='W-SUN'): -0.0066724332600349955,
 Timestamp('2003-11-23 00:00:00', freq='W-SUN'): 0.016811490747702815,
 Timestamp('2003-11-30 00:00:00', freq='W-SUN'): 0.0,
 Timestamp('2003-12-07 00:00:00', freq='W-SUN'): 0.013115992376166272,
 Timestamp('2003-12-14 00:00:00', freq='W-SUN'): -0.002473170330185461,
 Timestamp('2003-12-21 00:00:00', freq='W-SUN'): 0.008364702561221527,
 Timestamp('20

### Cross validation

In [38]:
# model = torch.exp(torch.matmul(x, a)) / (torch.exp (torch.matmul(x, a)) + torch.exp(torch.matmul(x, b)))

def train_epoch(dataloader):
    train_loss,train_correct=0.0,0
    for data in dataloader:

        x = data[:,:-1]
        y = data[:,-1]

        y_output = torch.exp(torch.matmul(x, a)) / (torch.exp (torch.matmul(x, a)) + torch.exp(torch.matmul(x, b)))
        loss = loss_fn(y_output, y)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item() * x.size(0)

    return train_loss
  
def valid_epoch(dataloader):

    valid_loss, val_correct = 0.0, 0
    for data in dataloader:

        x = data[:,:-1]
        y = data[:,-1]

        y_output = torch.exp(torch.matmul(x, a)) / (torch.exp (torch.matmul(x, a)) + torch.exp(torch.matmul(x, b)))

        loss = loss_fn(y_output, y)

        valid_loss+=loss.item() * x.size(0)

    return valid_loss

In [39]:
history = {'train_loss': [], 'test_loss': []}
 
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    
    for epoch in range(n_epochs):
        train_loss=train_epoch(train_loader)
        test_loss=valid_epoch(test_loader)

        train_loss = train_loss / len(train_loader.sampler)
        test_loss = test_loss / len(test_loader.sampler)

        if epoch % 1 == 0:
            print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f}".format(epoch + 1,n_epochs,train_loss,test_loss))
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)

Fold 1


NameError: ignored

In [ ]:
print(a, b)

## Build efficient frontier

In [ ]:
with torch.no_grad():
  y_test = torch.exp(torch.matmul(X_tensor, a)) / (torch.exp (torch.matmul(X_tensor, a)) + torch.exp(torch.matmul(X_tensor, b)))
  print(y_test)

### Build ML Portfolio

In [ ]:
prob = pd.DataFrame(y_test).astype("float")
display(prob)
rolling_prob = prob.rolling(25).mean().iloc[-1]
display(rolling_prob.to_numpy()[0])

In [ ]:
def calculate_ml_portfolio_weights(x, k):
  return 0 if x < k else 1

for k in np.arange(0, 1, 0.1):
  print(calculate_ml_portfolio_weights(rolling_prob.to_numpy()[0], k))

In [ ]:
k = 0.5
calculate_ml_portfolio_weights_lambda = lambda x: 0 if x < k else 1
calculate_ml_portfolio_weights = np.vectorize(calculate_ml_portfolio_weights_lambda)
# vfunc(x)
# calculate_ml_portfolio_weights = functorch.vmap(ml_portfolio_weights, out_dims=1)
# forecast = 
# portfolio_weights = calculate_ml_portfolio_weights(y_test.numpy())
# print(portfolio_weights)

portfolio_weights = y_test.apply_(calculate_ml_portfolio_weights_lambda)
print(portfolio_weights)

In [ ]:
Xt = torch.from_numpy(to_X_train_features(low_risk, high_risk).T[-1])
Xt

In [ ]:
with torch.no_grad():
  y_test = torch.exp(torch.matmul(Xt, a)) / (torch.exp (torch.matmul(Xt, a)) + torch.exp(torch.matmul(Xt, b)))
  print(y_test)

In [ ]:
k = 0.5
calculate_ml_portfolio_weights_lambda = lambda x: 0 if x < k else 1
calculate_ml_portfolio_weights = np.vectorize(calculate_ml_portfolio_weights_lambda)

portfolio_weights = y_test.apply_(calculate_ml_portfolio_weights_lambda)
print(portfolio_weights)

# MV Portfolio

In [ ]:
def add_daily_return(market_data):
    market_data["Pct Return"]  = market_data['Close'].pct_change()

add_daily_return(high_risk)
add_daily_return(low_risk)

In [ ]:
high_risk

In [ ]:
high_risk_return_annual = high_risk["Pct Return"].mean() * trading_days_in_year
low_risk_return_annual = low_risk["Pct Return"].mean() * trading_days_in_year
print(high_risk_return_annual)
print(low_risk_return_annual)

In [ ]:
high_risk_var_daily = high_risk["Pct Return"].var()
low_risk_var_daily = low_risk["Pct Return"].var()
print(high_risk_var_daily)
print(low_risk_var_daily)

## Build data for high and low risk

In [ ]:
mv_data = pd.DataFrame(data={'high': high_risk['Close'], 'low':low_risk['Close']})
mv_data

In [ ]:
def get_annual_sample_return_and_covariance(data):
    daily_return = data.pct_change()
    annual_return = daily_return.mean() * trading_days_in_year
    # daily_covariance = data.cov()
    daily_covariance = daily_return.cov()
    annual_covariance = daily_covariance * trading_days_in_year
    return annual_return, annual_covariance

In [ ]:
r, cov = get_annual_sample_return_and_covariance(mv_data)
display(r)
display(cov)

In [ ]:
def get_sample_return_and_covariance(data):
    daily_return = data.pct_change().mean()
    daily_covariance = data.pct_change().cov()
    return daily_return, daily_covariance

In [ ]:
r, cov = get_sample_return_and_covariance(mv_data)
display(r)
display(cov)

## Optimization using linear programming

(Reference: https://www.kaggle.com/code/vijipai/lesson-5-mean-variance-optimization-of-portfolios)


In [ ]:
TERMINATION = 10**-9

In [ ]:
#function obtains maximal return portfolio using linear programming

def MaximizeReturns(MeanReturns, PortfolioSize):
    
    #dependencies
    from scipy.optimize import linprog
    import numpy as np
    
    c = (np.multiply(-1, MeanReturns))
    A = np.ones([PortfolioSize,1]).T
    b=[1] 
    res = linprog(c, A_ub = A, b_ub = b, bounds = (0,1), method = 'simplex') 
    
    return res

In [ ]:
#function obtains minimal risk portfolio 

from scipy import optimize 

def MinimizeRisk(CovarReturns, PortfolioSize):
    
    def  f(x, CovarReturns):
        func = np.matmul(np.matmul(x, CovarReturns), x.T) 
        return func

    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    xinit=np.repeat(0.1, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])

    opt = optimize.minimize (f, x0 = xinit, args = (CovarReturns),  bounds = bnds, \
                             constraints = cons, tol = TERMINATION)
    
    return opt

In [ ]:
def print_min_variance_portfolio(mean_returns, cov_returns):
    number_of_assets = len(mean_returns)
    result = MinimizeRisk(cov_returns, number_of_assets)

    print()
    minRiskWeights = result.x
    minRiskExpPortfolioReturn = np.matmul(mean_returns.T, minRiskWeights)
    print("Expected Return of Minimum Risk Portfolio:  %7.6f" % minRiskExpPortfolioReturn)
    minRisk = np.matmul(np.matmul(minRiskWeights, cov_returns), minRiskWeights.T) 
    print("Variance of Minimum Risk Portfolio : %7.6f" % minRisk)
    print("S.D. of Minimum Risk Portfolio : %7.6f" % np.sqrt(minRisk))
    threshold = 1e-3
    print("Weights (showing only those > %.6f): " % threshold)
    for i in range(0, number_of_assets):
        if result.x[i] > threshold:
            print(f"{mean_returns.index[i]}\t{result.x[i]:.6f}")
    print('Assets Considered:')
    print(mean_returns.index.to_numpy())

In [ ]:
#function obtains Minimal risk and Maximum return portfolios

#dependencies
import numpy as np
from scipy import optimize 

def MinimizeRiskConstr(MeanReturns, CovarReturns, PortfolioSize, R):
    
    def  f(x,CovarReturns):
         
        func = np.matmul(np.matmul(x,CovarReturns ), x.T)
        return func

    def constraintEq(x):
        AEq=np.ones(x.shape)
        bEq=1
        EqconstraintVal = np.matmul(AEq,x.T)-bEq 
        return EqconstraintVal
    
    def constraintIneq(x, MeanReturns, R):
        AIneq = np.array(MeanReturns)
        bIneq = R
        IneqconstraintVal = np.matmul(AIneq,x.T) - bIneq
        return IneqconstraintVal
    

    xinit=np.repeat(0.1, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq},
            {'type':'ineq', 'fun':constraintIneq, 'args':(MeanReturns,R) })
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])

    opt = optimize.minimize (f, args = (CovarReturns), method ='trust-constr',  \
                        x0 = xinit,   bounds = bnds, constraints = cons, tol = TERMINATION)
    
    return  opt

In [ ]:
print_min_variance_portfolio(r, cov)

In [ ]:
#Maximal expected portfolio return computation for the k-portfolio
result1 = MaximizeReturns(r, 2)
maxReturnWeights = result1.x
maxExpPortfolioReturn = np.matmul(r.T, maxReturnWeights)
print("Maximal Expected Portfolio Return:   %7.6f" % maxExpPortfolioReturn )

In [ ]:
#expected portfolio return computation for the minimum risk k-portfolio 
result2 = MinimizeRisk(cov, 2)
minRiskWeights = result2.x
minRiskExpPortfolioReturn = np.matmul(r.T, minRiskWeights)
print("Expected Return of Minimum Risk Portfolio:  %7.6f" % minRiskExpPortfolioReturn)

In [ ]:
#compute efficient set for the maximum return and minimum risk portfolios
increment = 0.000001
low = minRiskExpPortfolioReturn
high = maxExpPortfolioReturn

#initialize optimal weight set and risk-return point set
xOptimal =[]
minRiskPoint = []
expPortfolioReturnPoint =[]

#repeated execution of function MinimizeRiskConstr to determine the efficient set 
while (low < high):
    
    result3 = MinimizeRiskConstr(r, cov, 2, low)
    xOptimal.append(result3.x)
    expPortfolioReturnPoint.append(low)
    low = low+increment
    
#gather optimal weight set    
xOptimalArray = np.array(xOptimal)

#obtain annualized risk for the efficient set portfolios 
#for trading days = 251
minRiskPoint = np.diagonal(np.matmul((np.matmul(xOptimalArray,cov)),\
                                     np.transpose(xOptimalArray)))
riskPoint =   np.sqrt(minRiskPoint*trading_days_in_year) 

#obtain expected portfolio annualized return for the 
#efficient set portfolios, for trading days = 251
retPoint = trading_days_in_year*np.array(expPortfolioReturnPoint) 

#display efficient set portfolio parameters
print("Size of the  efficient set:", xOptimalArray.shape )
print("Optimal weights of the efficient set portfolios: \n", xOptimalArray)
print("Annualized Risk and Return of the efficient set portfolios: \n", \
                                                np.c_[riskPoint, retPoint])

In [ ]:
#Graph Efficient Frontier
import matplotlib.pyplot as plt

NoPoints = riskPoint.size

colours = "green"
area = np.pi*3

plt.title('Efficient Frontier for MV Portfolio')
plt.xlabel('Annualized Risk(%)')
plt.ylabel('Annualized Expected Portfolio Return(%)' )
plt.scatter(riskPoint, retPoint, s=area, c=colours, alpha =0.5)
plt.show()

# Naive Portfolio

In [ ]:
naive_risks = []
naive_returns = []

for x in np.arange(0, 1, 0.01):
  weights = [x, 1-x]
  risk = np.matmul((np.matmul(weights,cov)),np.transpose(weights)) * trading_days_in_year
  naive_risks.append(np.sqrt(risk))

  #obtain expected portfolio annualized return for the 
  #efficient set portfolios, for trading days = 251
  ret = trading_days_in_year*(np.matmul(weights,r))
  naive_returns.append(ret)

#display efficient set portfolio parameters
# print("Size of the  efficient set:", xOptimalArray.shape )
# print("Optimal weights of the efficient set portfolios: \n", xOptimalArray)
print("Annualized Risk and Return of the efficient set portfolios: \n", \
                                                np.c_[naive_risks, naive_returns])

In [ ]:
NoPoints = len(naive_risks)

colours = "blue"
area = np.pi*3

plt.title('Efficient Frontier for Naive Portfolio')
plt.xlabel('Annualized Risk(%)')
plt.ylabel('Annualized Expected Portfolio Return(%)' )
plt.scatter(naive_risks, naive_returns, s=area, c=colours, alpha =0.5)
plt.show()

# Combined Graph

In [ ]:
NoPoints = riskPoint.size

colours = "blue"
area = np.pi*3


fig = plt.figure()
ax1 = fig.add_subplot(111)

plt.title('Efficient Frontier for MV and Naive Portfolio')
plt.xlabel('Annualized Risk(%)')
plt.ylabel('Annualized Expected Portfolio Return(%)' )
ax1.scatter(riskPoint, retPoint, s=area, c="green", alpha =0.5)
ax1.scatter(naive_risks, naive_returns, s=area, c="blue", alpha =0.5)
# plt.xlim(riskPoint.min(), riskPoint.max())
plt.show()

In [ ]:
cov

In [ ]:
print(high_risk["Close"].var())
print(high_risk["Close"].var() * trading_days_in_year)
print(high_risk["Close"].pct_change().var())
print(high_risk["Close"].pct_change().var() * trading_days_in_year)
print(np.sqrt(high_risk["Close"].pct_change().var() * trading_days_in_year))